# Mission Statement

In this project, I will explore a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. 

The aim of this project is to clean the data and analyze the user car listings on this site.


### Data Dictionary

The data dictionary provided with data is as follows:

    dateCrawled - When this ad was first crawled. All field-values are taken from this date.
    name - Name of the car.
    seller - Whether the seller is private or a dealer.
    offerType - The type of listing
    price - The price on the ad to sell the car.
    abtest - Whether the listing is included in an A/B test.
    vehicleType - The vehicle Type.
    yearOfRegistration - The year in which which year the car was first registered.
    gearbox - The transmission type.
    powerPS - The power of the car in PS.
    model - The car model name.
    kilometer - How many kilometers the car has driven.
    monthOfRegistration - The month in which which year the car was first registered.
    fuelType - What type of fuel the car uses.
    brand - The brand of the car.
    notRepairedDamage - If the car has a damage which is not yet repaired.
    dateCreated - The date on which the eBay listing was created.
    nrOfPictures - The number of pictures in the ad.
    postalCode - The postal code for the location of the vehicle.
    lastSeenOnline - When the crawler saw this ad last online.
    
    
# Import and Explore the Data

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv",encoding='Latin-1')

In [2]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

After viewing the first few rows of the dataset and the general info about this dataset.

The dataset contains 20 columns, most of which are strings.

Almost every non-numeric column is in German, which will be difficult to understand as a non-German speaker. A few of these columns can be converted to English to make the analysis more meaningful. 

A few numeric values (such as odometer) will need to be cleaned and have extra characters removed to work with numeric data. Date values will need to be cleaned to DateTime values to work with easier as well.

There are 5 columns that contain null values. We will also  explore these null values and determine whether to remove these rows or replace them with other values. No row has > 20% null values, which provides us with flexibility.

The column headers also are noted to use camelcase. We will convert this to snakecase and reword some of the column names to be more descriptive

# Creating Descriptive Column Headers

Here I converted camelcase headers to snakecase to quickly access the columns later in my analysis. I also changed several headers to a more accurate header according to the data dictionary

    yearOfRegistration to registration_year
    monthOfRegistration to registration_month
    notRepairedDamage to unrepaired damage
    dateCreated to ad_created
    nrOfPictures to num_photos

In [4]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [5]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
                   'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
                   'odometer', 'registration_month', 'fuel_type', 'brand',
                   'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
                   'last_seen']

# Cleaning the Data

I will first explore the data to determine what cleaning tasks need to be done.

Initially I will screen for:
1. Text columns where all or almost all values are the same. These can often be dropped as they don't provide useful information for analysis. 
2. Numeric data stored as text that needs to be cleaned and converted.

In [6]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 15:49:30,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Analyzing these descriptive statistics, we find several columns that could be removed.

seller, offer_type, ab_test, gearbox, and unrepaired_damage are columns that only have 2 unique values in the column. We will explore these to determine if they can be removed. num_photos contain only 0s and this column will likely be removed.

vehicle_type, and brand will be further investigated to check if there are similar entries that are listed as unique due to spelling or capitalization. 

registration_year and power_ps are numeric columns that will be investigated for outliers or erroneous values. For example, registration year's max entry is 9999. 

price and odometer are stored as text but are numeric values. These columns will be cleaned and converted to numeric. 
date_crawled and ad_created will be converted to DateType as well.

## Finding useless columns

Below we will analyze seller, offer_type, ab_test, gearbox, unrepaired_damage, and num_photos to determine if these columns all have 1 value or close to that

##### Seller
Seller has only only one row that is not listed as 'privat'. As a result, we will remove this column from our dataset.

In [7]:
autos['seller'].value_counts(dropna=False)

privat        49999
gewerblich        1
Name: seller, dtype: int64

##### offer_type

offer_type has only one row not listed as 'Angebot'. As a result, we will remove this column

In [8]:
autos['offer_type'].value_counts(dropna=False)

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

##### ab_test 

ab_test has 2 values, test and control, each with roughly 25000 values. This column will not be modified or removed.

In [9]:
autos['ab_test'].value_counts(dropna=False)

test       25756
control    24244
Name: ab_test, dtype: int64

##### gearbox
gearbox has 2 different values, manuell and automatik. This row must stay because it provides insightful data. However, we will translate these values to english

In [10]:
autos['gearbox'].value_counts(dropna=False)

manuell      36993
automatik    10327
NaN           2680
Name: gearbox, dtype: int64

In [11]:
autos['gearbox'].fillna('unknown',inplace = True)
autos['gearbox'].replace("manuell",'manual',inplace=True)
autos['gearbox'].replace('automatik','automatic',inplace=True)

##### unrepaired_damage

unrepaired_damage has 2 different values, nein and ja. This row must stay because it provides insightful data. However, we will translate these values to english

In [12]:
autos['unrepaired_damage'].value_counts(dropna=False)

nein    35232
NaN      9829
ja       4939
Name: unrepaired_damage, dtype: int64

In [13]:
autos['unrepaired_damage'].fillna('unknown',inplace = True)
autos['unrepaired_damage'].replace("nein",'no',inplace=True)
autos['unrepaired_damage'].replace('ja','yes',inplace=True)

##### num_photos
num_photos only has one value - 0. This column will be removed.

In [14]:
autos['num_photos'].value_counts(dropna=False)

0    50000
Name: num_photos, dtype: int64

In [15]:
autos.drop(labels = ['seller','offer_type','num_photos'],axis=1,inplace=True)

## Converting numerics stored as text to numerics

price and odometer are numeric values that are stored as text. These columns will be cleaned and converted to the appropriate numeric type.

For price, dollar signs and commas must be removed. For odometer, commas and km must be removed. After removing these characters, these values will be converted to integers

In [18]:
autos['price']=autos['price'].str.replace('$','').str.replace(',','').astype(int)

autos['price'].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int32

In [19]:
autos['odometer'] = autos['odometer'].str.replace(',','').str.replace('km','').astype(int)
autos.rename({'odometer':'odometer_km'},axis = 1,inplace = True)
autos['odometer_km'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int32

### Exploring Odometer and Price

These two columns will continue to be explored now that they are the correct datatype. We will look specifically for outliers that we may want to remove.

#### price

Price has 2357 unique values for 50,000 entries. This may because people often round to the nearest hundred.

There are 1,421 vehicles that have a price of $0. This is likely suspicious data as people rarely give away their cars for free. Given that these entries account for less than 3% of the listings, we can consider removing these values.

The max value however is 100 million dollars, which seems highly suspicious. The owner of the car either highly values his car or it is an error. We will the higher values further.

The mean price of a vehicle is 9,840 dollars, while the price in the 75th percentile is 7,200 dollars. The standard deviation is also very high at 484,381 dollars. The outliers are significantly impacting these values.

In [20]:
autos['price']

autos['price'].unique().shape

(2357,)

In [21]:
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [22]:
autos['price'].value_counts(dropna=False).head(20)

0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

In [23]:
autos['price'].value_counts(dropna=False)[0]/autos.shape[0]

0.02842

In [24]:
autos['price'].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [25]:
autos['price'].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

Looking the most expensive listings, 14 listings are above 350,000 dollars. Prices increase steadily to 350,000 dollars, but then immediately jump to close to 1 million dollars and greater. We will keep all listings below 350,000 dollars.

Looking at the least expensive listings, there a number of listings below 35 dollars that are not 0 dollars. Given that eBay is an auction websites, these could be legitimate bids in the early stages of the bidding process. Because of this, we will keep values that are greater than 0 dollars.

In [26]:
autos = autos[autos['price'].between(1,350000)]
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

Now our summary statistics look much more realistic after removing outliers.

#### odometer_km

There are 13 unique values for mileage. Looking at the value_counts for this column, the 13 values are values rounded to the thousands. This likely indicates they could not manually enter the exact value, but had to pick from a list of options.

Roughly 80% of the vehicles listed have greater than 100,000 miles.

In [27]:
autos['odometer_km'].unique().shape

(13,)

In [28]:
autos['odometer_km'].describe()

count     48565.000000
mean     125770.101925
std       39788.636804
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [29]:
autos['odometer_km'].value_counts()

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
5000        836
40000       815
30000       780
20000       762
10000       253
Name: odometer_km, dtype: int64

In [30]:
print(round((autos['odometer_km'].between(100000,150000).sum()/autos.shape[0])*100,0), " % of vehicles in this dataset have over 100,000 miles")

79.0  % of vehicles in this dataset have over 100,000 miles


## Exploring the date columns

There are 5 columns that represent date values. Some of these were created by the crawler, and some from the website itself.

- date_crawled - added by crawler
- last_seen - added by crawler
- ad_created - from the website
- registration_month - from the website
- registration_year - from the website

date_crawled, last_seen, and ad_created columns are all identified as string values. We need to convert these columns into a numerical representation to understand it quantitatively. Lets first take a look at these 3 columns.

The first 10 characters represent the day, and the last 8 characters represent the time. Lets extract the date values, and get a distribution by date.

#### date_crawled
The site was crawled daily over roughly a one month period from March 5th 2016 - April 7th 2016. The number of listings crawled on each day is roughly consistent as well

In [31]:
autos[['date_crawled','last_seen','ad_created']].head()

,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


In [32]:
autos['date_crawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

#### last_seen

This value is the last date the crawler recording seeing any listing. This allows us to determine on what day a listing was removed, likely because the car was sold. 

This percentage is relatively consistent for most dates that aren't at the beginning or end of this period (roughly 1-3%). The last three days contain a disproportionate amount of 'last seen' values. These values are about 6-10x the values from the previous days. This is likely because this is when the crawling period ended, and not because about 50% of the sales occured in the last 3 days.

In [33]:
autos['last_seen'].str[:10].value_counts(normalize=True,dropna=False).sort_values()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-18    0.007351
2016-03-08    0.007413
2016-03-13    0.008895
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-14    0.012602
2016-03-27    0.015649
2016-03-19    0.015834
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-26    0.016802
2016-03-23    0.018532
2016-03-25    0.019211
2016-03-24    0.019767
2016-03-21    0.020632
2016-03-20    0.020653
2016-03-28    0.020859
2016-03-22    0.021373
2016-03-29    0.022341
2016-04-01    0.022794
2016-03-12    0.023783
2016-03-31    0.023783
2016-04-04    0.024483
2016-03-30    0.024771
2016-04-02    0.024915
2016-04-03    0.025203
2016-03-17    0.028086
2016-04-05    0.124761
2016-04-07    0.131947
2016-04-06    0.221806
Name: last_seen, dtype: float64

#### ad_created

Several of these posts were created as far back as June 2015 and are still listed. However, most fall within 1-2 months of the listing date. The postings created in March and April 2016 comprise most of the listings in this data.

In [34]:
autos['ad_created'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

## Odd values in registration_year

There are odd values in some of these columns that do not make logical sense. For example, in registration year:

- min val is 1000, before cars were invented
- max val is 9999, we are in 2020.

Because this dataset is from 2016, any registration year above 2016 is inaccurate. Determining an earliest valid year is arbitrary. We will example the number of entries by decade to determine what years are incorrect.

In [35]:
autos['registration_year'] = autos['registration_year'].astype(int)

decade = {'before 1900': 0,
          '1900s':0,
          '1910s':0,
          '1920s':0,
          '1930s':0,
          '1940s':0,
          '1950s':0,
          '1960s':0,
          '1970s':0,
          '1980s':0,
          '1990s':0,
          '2000s':0,
          '2010-2016':0,
          'after 2016':0,
          'null': 0
}
registration_year = autos['registration_year']


for value in registration_year:
    if value < 1900:
        decade['before 1900'] += 1
    elif 1900 <= value < 1910:
        decade['1900s'] += 1
    elif 1910 <= value < 1920:
        decade['1910s'] += 1
    elif 1920 <= value < 1930:
        decade['1920s'] += 1
    elif 1930 <= value < 1940:
        decade['1930s'] += 1
    elif 1940 <= value < 1950:
        decade['1940s'] += 1
    elif 1950 <= value < 1960:
        decade['1950s'] += 1
    elif 1960 <= value < 1970:
        decade['1960s'] += 1
    elif 1970 <= value < 1980:
        decade['1970s'] += 1
    elif 1980 <= value < 1990:
        decade['1980s'] += 1
    elif 1990 <= value < 2000:
        decade['1990s'] += 1
    elif 2000 <= value < 2010:
        decade['2000s'] += 1
    elif 2010 <= value <= 2016:
        decade['2010-2016'] += 1
    elif value > 2016:
        decade['after 2016'] += 1
    else:
        decade['null']+=1
        
decade

{'before 1900': 5,
 '1900s': 0,
 '1910s': 5,
 '1920s': 2,
 '1930s': 9,
 '1940s': 4,
 '1950s': 27,
 '1960s': 163,
 '1970s': 283,
 '1980s': 804,
 '1990s': 11921,
 '2000s': 25863,
 '2010-2016': 7600,
 'after 2016': 1879,
 'null': 0}

In [36]:
(~autos['registration_year'].between(1900,2016)).sum()/autos.shape[0]

0.038793369710697

From our frequency table, we see that only 5 cars were before 1900 and 1,879 are "after" 2016. Given that this makes up roughly 4% of our data, we can remove these rows.

Our frequency table also shows us that majority of our cars were registered in the past 3 decades (1990s-2010s)

In [40]:
autos = autos[autos['registration_year'].between(1900,2016)]

## Exploring the data by brand

We will explore the top brands in this dataset, which we will limit to having more than 5% of total listings.

German manufacturers represent 4 of the top 5 brands and almost 50% of the overall listings.
Volkswagen is the most popular by far, with 21% of the listings being for VWs. It also has double the cars for sale than any other brand.m

In [49]:
autos['brand'].value_counts(dropna=False,normalize=True).head(10)

volkswagen       0.211264
bmw              0.110045
opel             0.107581
mercedes_benz    0.096463
audi             0.086566
ford             0.069900
renault          0.047150
peugeot          0.029841
fiat             0.025642
seat             0.018273
Name: brand, dtype: float64

In [52]:
brand_rank = autos['brand'].value_counts(dropna=False,normalize=True)
top_brands = brand_rank[brand_rank>.05].index
top_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

### Average Price of Top Brands


In [70]:
top_brand_price = {}

for brand in top_brands:
    brand_ds = autos[autos['brand']==brand]
    mean_price = brand_ds['price'].mean()
    top_brand_price[brand] = int(mean_price)

top_brand_price_df = pd.DataFrame.from_dict(top_brand_price,orient='index',columns=['Average Price'])
top_brand_price_df.sort_values(by='Average Price',ascending = False,inplace=True)
top_brand_price_df

,Average Price
audi,9336
mercedes_benz,8628
bmw,8332
volkswagen,5402
ford,3749
opel,2975


Of the top brands, there is an evident price gap. BMW, Mercedez_Benx, and Audo trade at a significantly higher price point. Opel and Ford are less expensive, and VW is in between. 

The medium price point may explain why it is the top sold brand - it is affordable but still not considered a "cheap" car.

### Mileage of Top Brands

In [77]:
top_brand_mileage_km = {}

for brand in top_brands:
    brand_ds = autos[autos['brand']==brand]
    mean_odom = brand_ds['odometer_km'].mean()
    top_brand_mileage_km[brand] = int(mean_odom)

top_brand_mileage_df = pd.DataFrame.from_dict(top_brand_mileage_km,orient='index',columns=['Average Mileage (km)'])
top_brand_mileage_df.sort_values(by='Average Mileage (km)',ascending = False,inplace=True)
top_brand_mileage_df['Average Price ($)']=top_brand_price_df
top_brand_mileage_df

,Average Mileage (km),Average Price ($)
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


The average mileage is consistent across brands, only differing by 10% from the top brand. The more expensive brands on average do have more mileage than the cheaper vehicles, but the difference is minimal.

### Price of a VW with Varying Mileage

Roughly 70% of all VWs for sale have mileage greater than 150,000. Let's examine what the average price of a vw is dependent on mileage

In [81]:
vw_data = autos[autos['brand']=='volkswagen']
vw_data['odometer_km'].value_counts(normalize= True)

150000    0.696208
125000    0.085682
100000    0.037822
90000     0.029102
80000     0.025350
60000     0.020990
70000     0.020888
50000     0.018657
30000     0.016731
5000      0.015514
40000     0.014500
20000     0.013689
10000     0.004867
Name: odometer_km, dtype: float64

In [89]:
mileage_ranges = vw_data['odometer_km'].value_counts().index

price_by_mileage_vw = {}

for mil_range in mileage_ranges:
    range_df = vw_data[vw_data['odometer_km']==mil_range]
    avg_price = range_df['price'].mean()
    price_by_mileage_vw[mil_range] = avg_price
    
vw_price_by_mileage = pd.DataFrame.from_dict(price_by_mileage_vw,orient='index',columns=['Average Price by Mileage'])
vw_price_by_mileage.sort_index(axis=0,inplace = True,ascending=False)
vw_price_by_mileage

,Average Price by Mileage
150000,3483.571803
125000,6027.475740
100000,7978.981233
90000,8739.529617
80000,10722.832000
70000,11472.611650
60000,12659.797101
50000,13296.559783
40000,15678.615385
30000,15213.412121


As expected, the fewer miles on your vehicle, the more expensive the vehicle is. Once your vehicle exceeds 150,000 miles, the price of your car drops in half. 

### Automatic vs. Manual

In [92]:
autos['gearbox'].value_counts(dropna=False,normalize=True).head(10)

,Average Price by Gear Type
150000,NaN
125000,NaN
100000,NaN
90000,NaN
80000,NaN
70000,NaN
60000,NaN
50000,NaN
40000,NaN
5000,NaN


In [97]:
gear_type = autos['gearbox'].value_counts().index

price_by_gearbox = {}

for gear in gear_type:
    gear_df = autos[autos['gearbox']==gear]
    avg_price = gear_df['price'].mean()
    price_by_gearbox[gear] = avg_price
    
price_by_gear_type = pd.DataFrame.from_dict(price_by_gearbox,orient='index',columns=['Average Price by Gear Type'])
price_by_gear_type.sort_values(by='Average Price by Gear Type',inplace=True,ascending=False)
price_by_gear_type

,Average Price by Gear Type
automatic,10972.718547
manual,4716.709175
unknown,3392.476303


Automatic vehicles are significantly more expensive than manual vehicles, roughly double the value. Let's examine if automatic vehicles are produced more commonly by luxury automakers

In [119]:
count_by_auto = {}

for brand in top_brands:
    brand_ds = autos[autos['brand']==brand]
    auto_ds = brand_ds[brand_ds['gearbox']=='automatic']
    manual_ds = brand_ds[brand_ds['gearbox']=='manual']
    auto_count = len(auto_ds)
    auto_pct = round((len(auto_ds)/len(brand_ds))*100,1)
    manual_count = len(manual_ds)
    manual_pct = round((len(manual_ds)/len(brand_ds))*100,1)
    count_by_auto[brand] = [auto_count,manual_count,auto_pct,manual_pct]
    
count_by_auto

{'volkswagen': [1296, 8077, 13.1, 81.9],
 'bmw': [1716, 3275, 33.4, 63.8],
 'opel': [511, 4264, 10.2, 84.9],
 'mercedes_benz': [2477, 1881, 55.0, 41.8],
 'audi': [1330, 2560, 32.9, 63.4],
 'ford': [258, 2849, 7.9, 87.3]}

In [120]:
count_by_auto_ds = pd.DataFrame.from_dict(count_by_auto).T
count_by_auto_ds.columns = ['Automatic','Manual','Automatic_Pct','Manual_Pct']
count_by_auto_ds['Price'] = top_brand_price_df
count_by_auto_ds

,Automatic,Manual,Automatic_Pct,Manual_Pct,Price
volkswagen,1296.0,8077.0,13.1,81.9,5402
bmw,1716.0,3275.0,33.4,63.8,8332
opel,511.0,4264.0,10.2,84.9,2975
mercedes_benz,2477.0,1881.0,55.0,41.8,8628
audi,1330.0,2560.0,32.9,63.4,9336
ford,258.0,2849.0,7.9,87.3,3749


Over 80% of all VWs, Opels, and Fords are manual, and these 3 vehicles have the lowest price point, which likely explains why manual vehicles are much more cheap than automatic. 